# Pacific Decadal Oscillation - PDO index

Mantua et al. (1997)

> The leading EOF of monthlySST anomalies over the North Pacific (after removing the global mean SST anomaly) and its associated PC time series are termed the Pacific Decadal Oscillation (PDO)

1. compute monthly global mean SST, deseasonalize it

2. North Pacific monthly output fields

    2.1. create monthly field, determine extend of grid, limit all coordinates appropriately, save as single file
        
        a) North of 0 deg N
        
        b) North of 20 deg N
    
    2.2. deseasonalize and remove global mean, deseasonalized SST, remove mean

3. EOF analysis

maybe use ocn rect for it

In [ ]:
import os
import sys
sys.path.append("..")
import scipy as sp
import numpy as np
import xarray as xr
import seaborn as sns
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
from OHC import t2da, t2ds
from SST import SST_index, EOF_SST_analysis
from maps import map_robinson, map_eq_earth, rect_polygon, regr_map
from grid import find_array_idx
from paths import path_results, path_samoc, file_ex_ocn_ctrl, file_ex_ocn_rect, file_ex_ocn_lpi, file_HadISST
from regions import boolean_mask, SOM_area, global_ocean, gl_ocean_rect, gl_ocean_low, mask_box_in_region
from plotting import shifted_color_map, discrete_cmap
from timeseries import IterateOutputCESM, lowpass, chebychev, notch, deseasonalize
from xr_DataArrays import xr_AREA
from xr_regression import xr_lintrend, xr_linear_trends_2D, xr_linear_trend, ocn_field_regression, lag_linregress_3D, xr_quadtrend

### 1. global mean SST temperature series

HadISST 60S-60N time series is generated in SST_obs.ipynb

In [ ]:
# SST_gm_ctrl      = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_ctrl.nc'     , decode_times=False)
# SST_gm_rcp       = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rcp.nc'      , decode_times=False)
SST_gm_ctrl = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rect_ctrl.nc', decode_times=False)
SST_gm_rcp  = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_rect_rcp.nc' , decode_times=False)
SST_gm_lpd  = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_lpd.nc'      , decode_times=False)
SST_gm_lpi  = xr.open_dataarray(f'{path_results}/SST_global_mean_monthly_lpi.nc'      , decode_times=False)

SST_xm_ctrl = xr.open_dataarray(f'{path_results}/SST_60S_60N_mean_monthly_rect_ctrl.nc', decode_times=False)
SST_xm_rcp  = xr.open_dataarray(f'{path_results}/SST_60S_60N_mean_monthly_rect_rcp.nc' , decode_times=False)
SST_xm_lpd  = xr.open_dataarray(f'{path_results}/SST_60S_60N_mean_monthly_lpd.nc'      , decode_times=False)
SST_xm_lpi  = xr.open_dataarray(f'{path_results}/SST_60S_60N_mean_monthly_lpi.nc'      , decode_times=False)
SST_xm_had  = xr.open_dataarray(f'{path_results}/SST_60S_60N_mean_monthly_had.nc'      , decode_times=False)

In [ ]:
# deseasonalize
SST_gm_ds_ctrl = deseasonalize(SST_gm_ctrl)
SST_gm_ds_rcp  = deseasonalize(SST_gm_rcp )
SST_gm_ds_lpd  = deseasonalize(SST_gm_lpd )
SST_gm_ds_lpi  = deseasonalize(SST_gm_lpi )
SST_xm_ds_ctrl = deseasonalize(SST_xm_ctrl)
SST_xm_ds_rcp  = deseasonalize(SST_xm_rcp )
SST_xm_ds_lpd  = deseasonalize(SST_xm_lpd )
SST_xm_ds_lpi  = deseasonalize(SST_xm_lpi )
SST_xm_ds_had  = deseasonalize(SST_xm_had )

In [ ]:
plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.plot([-1.5,2], [-1  ,2.5], c='k', lw=.5)
plt.plot([-1.5,2], [-1.5,2  ], c='k', lw=.5)
plt.plot([-1.5,2], [-0.5,3  ], c='k', lw=.5)
plt.plot([-1.5,2], [-2  ,1.5], c='k', lw=.5)
plt.xlim((-1,1.5))
plt.ylim((-1,1.5))
plt.scatter(SST_xm_ds_ctrl-SST_xm_ds_ctrl.mean(dim='time'), SST_gm_ds_ctrl-SST_gm_ds_ctrl.mean(dim='time')+.5, alpha=.1)
plt.scatter(SST_xm_ds_rcp -SST_xm_ds_rcp .mean(dim='time'), SST_gm_ds_rcp -SST_gm_ds_rcp .mean(dim='time')   , alpha=.1)
plt.scatter(SST_xm_ds_lpd -SST_xm_ds_lpd .mean(dim='time'), SST_gm_ds_lpd -SST_gm_ds_lpd .mean(dim='time')+1 , alpha=.1)
plt.scatter(SST_xm_ds_lpi -SST_xm_ds_lpi .mean(dim='time'), SST_gm_ds_lpi -SST_gm_ds_lpi .mean(dim='time')-.5, alpha=.1)
plt.scatter(SST_xm_ds_had -SST_xm_ds_had .mean(dim='time'), [-.9]*len(SST_xm_ds_had), alpha=.1)
plt.ylabel('global mean SST' , fontsize=14)
plt.xlabel('60S-60N mean SST', fontsize=14)
plt.tight_layout()

In [ ]:
plt.figure(figsize=(8,5))
plt.tick_params(labelsize=14)
plt.plot([-1.5,2], [-1  ,2.5], c='k', lw=.5)
plt.plot([-1.5,2], [-1.5,2  ], c='k', lw=.5)
plt.plot([-1.5,2], [-0.5,3  ], c='k', lw=.5)
plt.plot([-1.5,2], [-2  ,1.5], c='k', lw=.5)
plt.xlim((-.3,.3))
plt.ylim((-1.3,1.4))
plt.scatter(SST_xm_ds_ctrl-xr_quadtrend(SST_xm_ds_ctrl), SST_gm_ds_ctrl-xr_quadtrend(SST_gm_ds_ctrl)+.5, alpha=.1)
plt.scatter(SST_xm_ds_rcp -xr_quadtrend(SST_xm_ds_rcp ), SST_gm_ds_rcp -xr_quadtrend(SST_gm_ds_rcp )   , alpha=.1)
plt.scatter(SST_xm_ds_lpd -xr_quadtrend(SST_xm_ds_lpd ), SST_gm_ds_lpd -xr_quadtrend(SST_gm_ds_lpd )+1 , alpha=.1)
plt.scatter(SST_xm_ds_lpi -xr_quadtrend(SST_xm_ds_lpi ), SST_gm_ds_lpi -xr_quadtrend(SST_gm_ds_lpi )-.5, alpha=.1)
plt.scatter(SST_xm_ds_had -xr_quadtrend(SST_xm_ds_had ), [-1]*len(SST_xm_ds_had), alpha=.1)

plt.scatter(SST_xm_ds_lpd[-200*12:] -xr_quadtrend(SST_xm_ds_lpd[-200*12:] ),
            SST_gm_ds_lpd[-200*12:] -xr_quadtrend(SST_gm_ds_lpd[-200*12:] )+1.1 , alpha=.1)
plt.scatter(SST_xm_ds_lpi[-200*12:] -xr_quadtrend(SST_xm_ds_lpi[-200*12:] ),
            SST_gm_ds_lpi[-200*12:] -xr_quadtrend(SST_gm_ds_lpi[-200*12:] )-.6, alpha=.1)
plt.scatter(SST_xm_ds_had[-100*12:] -xr_quadtrend(SST_xm_ds_had[-100*12:] ),
            [-1.1]*(100*12), alpha=.1)
plt.ylabel('global mean SST' , fontsize=14)
plt.xlabel('60S-60N mean SST', fontsize=14)
plt.tight_layout()

In [ ]:
len(SST_xm_ds_had[-200*12:] )

In [ ]:
# lowpass deseasonalizing
f, ax = plt.subplots(1, 2, figsize=(12,5))
for i in range(2):
    ax[i].tick_params(labelsize=14)

ax[0].plot(SST_xm_ctrl.time[:120]/12   , SST_xm_ctrl   [:120])
ax[0].plot(SST_xm_ctrl.time[:120]/12   , SST_xm_ds_ctrl[:120])
ax[0].plot(SST_xm_rcp .time[:120]/12+11, SST_xm_rcp    [:120])
ax[0].plot(SST_xm_rcp .time[:120]/12+11, SST_xm_ds_rcp [:120])
ax[0].plot(SST_xm_rcp .time[:120]/12+11, SST_xm_ds_rcp [:120])

ax[1].plot(SST_xm_ctrl.time/12    , SST_xm_ctrl   )
ax[1].plot(SST_xm_ctrl.time/12    , SST_xm_ds_ctrl)
ax[1].plot(SST_xm_rcp .time/12+220, SST_xm_rcp    )
ax[1].plot(SST_xm_rcp .time/12+220, SST_xm_ds_rcp )
ax[1].plot(SST_xm_had .time/365   , SST_xm_had    )
ax[1].plot(SST_xm_had .time/365   , SST_xm_ds_had )
plt.tight_layout()
plt.savefig(f'{path_results}/SST/SST_deseasonalizing_global_mean')

In [ ]:
plt.figure(figsize=(12,5))
plt.tick_params(labelsize=14)

plt.plot(SST_gm_ctrl.time/12 +1950, SST_gm_ctrl   , c='C0', lw=.3, alpha=.5)
plt.plot(SST_gm_rcp .time/12 +2200, SST_gm_rcp    , c='C1', lw=.3, alpha=.5)
plt.plot(SST_gm_lpd .time/365+1350, SST_gm_lpd    , c='C2', lw=.3, alpha=.5)
plt.plot(SST_gm_lpi .time/365-1600, SST_gm_lpi    , c='C3', lw=.3, alpha=.5)
plt.plot(SST_gm_ctrl.time/12 +1950, SST_gm_ds_ctrl, c='C0')
plt.plot(SST_gm_rcp .time/12 +2200, SST_gm_ds_rcp , c='C1')
plt.plot(SST_gm_lpd .time/365+1350, SST_gm_ds_lpd , c='C2')
plt.plot(SST_gm_lpi .time/365-1600, SST_gm_ds_lpi , c='C3')

plt.plot(SST_xm_ctrl.time/12 +1950, SST_xm_ctrl   , c='C0', lw=.3, alpha=.5)
plt.plot(SST_xm_rcp .time/12 +2200, SST_xm_rcp    , c='C1', lw=.3, alpha=.5)
plt.plot(SST_xm_lpd .time/365+1350, SST_xm_lpd    , c='C2', lw=.3, alpha=.5)
plt.plot(SST_xm_lpi .time/365-1600, SST_xm_lpi    , c='C3', lw=.3, alpha=.5)
plt.plot(SST_xm_had .time/365+2350, SST_xm_had    , c='C4', lw=.3, alpha=.5)
plt.plot(SST_xm_ctrl.time/12 +1950, SST_xm_ds_ctrl, c='C0')
plt.plot(SST_xm_rcp .time/12 +2200, SST_xm_ds_rcp , c='C1')
plt.plot(SST_xm_lpd .time/365+1350, SST_xm_ds_lpd , c='C2')
plt.plot(SST_xm_lpi .time/365-1600, SST_xm_ds_lpi , c='C3')
plt.plot(SST_xm_had .time/365+2350, SST_xm_ds_had , c='C4')

plt.ylabel('global / 60S-60N mean SST [$^\circ$C]', fontsize=16)
plt.xlabel('time [years]', fontsize=16)

plt.tight_layout()

### 2.1. creating file with monthly SST fields in North Pacific

In [ ]:
f, ax = plt.subplots(3,3, figsize=(12,10))
for i, r in enumerate(['Pac_40S', 'Pac_Eq', 'Pac_20N']):
    latS = [-38, 0, 20][i]
    lonE = [300, 285, 255][i]
    Pac_MASK = mask_box_in_region(domain='ocn_rect', mask_nr=2, bounding_lats=(latS,68), bounding_lons=(110,lonE))
    Pac_MASK = Pac_MASK.where(Pac_MASK.t_lon+1/.6*Pac_MASK.t_lat<333,0)
    Pac_MASK.plot(ax=ax[0,i])
    Pac_MASK = mask_box_in_region(domain='ocn_low', mask_nr=2, bounding_lats=(latS,68), bounding_lons=(110,lonE))
    Pac_MASK.plot(ax=ax[1,i])
    Pac_MASK = mask_box_in_region(domain='ocn_had', mask_nr=2, bounding_lats=(latS,68), bounding_lons=(110,lonE))
    Pac_MASK.plot(ax=ax[2,i])
plt.tight_layout()

In [ ]:
SST_Pac_38S_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_Pac_38S_rect_ctrl.nc', decode_times=False)
SST_Pac_38S_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_Pac_38S_rect_rcp.nc' , decode_times=False)
SST_Pac_38S_had  = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_Pac_38S_had.nc'      , decode_times=False)

SST_Pac_Eq_ctrl  = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_Pac_Eq_rect_ctrl.nc' , decode_times=False)
SST_Pac_Eq_rcp   = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_Pac_Eq_rect_rcp.nc'  , decode_times=False)
SST_Pac_Eq_had   = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_Pac_Eq_had.nc'       , decode_times=False)

SST_Pac_20N_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_Pac_20N_rect_ctrl.nc', decode_times=False)
SST_Pac_20N_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_Pac_20N_rect_rcp.nc' , decode_times=False)
SST_Pac_20N_had  = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_Pac_20N_had.nc'      , decode_times=False)

### 2.2.1. deseasonalize monthly SST field

In [ ]:
%%time
# 6 min
SST_Pac_38S_ds_ctrl = deseasonalize(SST_Pac_38S_ctrl)
SST_Pac_38S_ds_rcp  = deseasonalize(SST_Pac_38S_rcp )
SST_Pac_38S_ds_had  = deseasonalize(SST_Pac_38S_had )
SST_Pac_Eq_ds_ctrl  = deseasonalize(SST_Pac_Eq_ctrl )
SST_Pac_Eq_ds_rcp   = deseasonalize(SST_Pac_Eq_rcp  )
SST_Pac_Eq_ds_had   = deseasonalize(SST_Pac_Eq_had  )
SST_Pac_20N_ds_ctrl = deseasonalize(SST_Pac_20N_ctrl)
SST_Pac_20N_ds_rcp  = deseasonalize(SST_Pac_20N_rcp )
SST_Pac_20N_ds_had  = deseasonalize(SST_Pac_20N_had )

In [ ]:
SST_Pac_Eq_ctrl[:120,100,100].plot()
SST_Pac_Eq_ds_ctrl[:120,100,100].plot()

the initial bias due to the arbitrary beginning month is the reason we cut off two years of the data on either side

### 2.2.2. detrend with 60S-60N mean, des. SST

In [ ]:
# in the gm timeseries are some months of the last incomplete year 

SST_Pac_38S_ds_dt_ctrl = SST_Pac_38S_ds_ctrl - SST_xm_ds_ctrl[:-7]
SST_Pac_38S_ds_dt_rcp  = SST_Pac_38S_ds_rcp  - SST_xm_ds_rcp[:-1]
SST_Pac_38S_ds_dt_had  = SST_Pac_38S_ds_had  - SST_xm_ds_had

SST_Pac_Eq_ds_dt_ctrl  = SST_Pac_Eq_ds_ctrl  - SST_xm_ds_ctrl[:-7]
SST_Pac_Eq_ds_dt_rcp   = SST_Pac_Eq_ds_rcp   - SST_xm_ds_rcp[:-1]
SST_Pac_Eq_ds_dt_had   = SST_Pac_Eq_ds_had   - SST_xm_ds_had

SST_Pac_20N_ds_dt_ctrl = SST_Pac_20N_ds_ctrl - SST_xm_ds_ctrl[:-7]
SST_Pac_20N_ds_dt_rcp  = SST_Pac_20N_ds_rcp  - SST_xm_ds_rcp[:-1]
SST_Pac_20N_ds_dt_had  = SST_Pac_20N_ds_had  - SST_xm_ds_had


### 2.2.3. remove remaining mean
this is probably not necessary

In [ ]:
SST_Pac_20N_ds_dt_ctrl[0,:,:].plot()

In [ ]:
SST_Pac_38S_ds_dt_dm_ctrl = SST_Pac_38S_ds_dt_ctrl - SST_Pac_38S_ds_dt_ctrl.mean('time')
SST_Pac_38S_ds_dt_dm_rcp  = SST_Pac_38S_ds_dt_rcp  - SST_Pac_38S_ds_dt_rcp .mean('time')
SST_Pac_38S_ds_dt_dm_had  = SST_Pac_38S_ds_dt_had  - SST_Pac_38S_ds_dt_had .mean('time')

SST_Pac_Eq_ds_dt_dm_ctrl  = SST_Pac_Eq_ds_dt_ctrl  - SST_Pac_Eq_ds_dt_ctrl .mean('time')
SST_Pac_Eq_ds_dt_dm_rcp   = SST_Pac_Eq_ds_dt_rcp   - SST_Pac_Eq_ds_dt_rcp  .mean('time')
SST_Pac_Eq_ds_dt_dm_had   = SST_Pac_Eq_ds_dt_had   - SST_Pac_Eq_ds_dt_had  .mean('time')

SST_Pac_20N_ds_dt_dm_ctrl = SST_Pac_20N_ds_dt_ctrl - SST_Pac_20N_ds_dt_ctrl.mean('time')
SST_Pac_20N_ds_dt_dm_rcp  = SST_Pac_20N_ds_dt_rcp  - SST_Pac_20N_ds_dt_rcp .mean('time')
SST_Pac_20N_ds_dt_dm_had  = SST_Pac_20N_ds_dt_had  - SST_Pac_20N_ds_dt_had .mean('time')


In [ ]:
SST_Pac_20N_ds_dt_dm_ctrl[0,:,:].plot()

### 3. EOF analysis

In [ ]:
SST_Pac_38S_ds_dt_dm_ctrl

In [ ]:
%%time
# 12 mins for all
# cut off 2 year on either end due to problems with the erroneous trend introduced by the filtering
for j, run in enumerate(['ctrl', 'rcp', 'had']):
    object_list = [[SST_Pac_38S_ds_dt_dm_ctrl, SST_Pac_Eq_ds_dt_dm_ctrl, SST_Pac_20N_ds_dt_dm_ctrl],
                   [SST_Pac_38S_ds_dt_dm_rcp , SST_Pac_Eq_ds_dt_dm_rcp , SST_Pac_20N_ds_dt_dm_rcp ],
                   [SST_Pac_38S_ds_dt_dm_had , SST_Pac_Eq_ds_dt_dm_had , SST_Pac_20N_ds_dt_dm_had ]
                  ][j]
    domain = ['ocn_rect', 'ocn_rect', 'ocn_had'][j]
    if j<2:
        continue
        domain = 'ocn_rect'
        lonE_list = [285, 285, 285]
    elif j==2:
        domain =  'ocn_had'
        lonE_list = lonE = [300, 285, 255]
    for i, r in enumerate(['Pac_38S', 'Pac_Eq', 'Pac_20N']):
        print(i)
        SST_object = object_list[i]
        fn = f'{path_results}/SST/SST_PDO_EOF_{r}_{run}.nc'
        latS = [-38,0,20][i]
        lonE = lonE_list[i]
        print(SST_object)
        Pac_MASK = mask_box_in_region(domain=domain, mask_nr=2, bounding_lats=(latS,68), bounding_lons=(110,lonE))
        print(xr_AREA(domain).where(Pac_MASK, drop=True))
        if j<2:
            Pac_area = xr_AREA(domain).where(Pac_MASK, drop=True)
        elif j==2:
            Pac_area = xr_AREA(domain)#.where(Pac_MASK, drop=True)
        eof, pc  = EOF_SST_analysis(xa=SST_object[24:-24], weights=Pac_area, fn=fn)

In [ ]:
Pac_38S_ctrl = xr.open_dataset(f'{path_results}/SST/SST_PDO_EOF_Pac_38S_ctrl.nc', decode_times=False)
Pac_38S_rcp  = xr.open_dataset(f'{path_results}/SST/SST_PDO_EOF_Pac_38S_rcp.nc' , decode_times=False)
Pac_38S_had  = xr.open_dataset(f'{path_results}/SST/SST_PDO_EOF_Pac_38S_had.nc' , decode_times=False)

Pac_Eq_ctrl  = xr.open_dataset(f'{path_results}/SST/SST_PDO_EOF_Pac_Eq_ctrl.nc' , decode_times=False)
Pac_Eq_rcp   = xr.open_dataset(f'{path_results}/SST/SST_PDO_EOF_Pac_Eq_rcp.nc'  , decode_times=False)
Pac_Eq_had   = xr.open_dataset(f'{path_results}/SST/SST_PDO_EOF_Pac_Eq_had.nc'  , decode_times=False)

Pac_20N_ctrl = xr.open_dataset(f'{path_results}/SST/SST_PDO_EOF_Pac_20N_ctrl.nc', decode_times=False)
Pac_20N_rcp  = xr.open_dataset(f'{path_results}/SST/SST_PDO_EOF_Pac_20N_rcp.nc' , decode_times=False)
Pac_20N_had  = xr.open_dataset(f'{path_results}/SST/SST_PDO_EOF_Pac_20N_had.nc' , decode_times=False)


In [ ]:
TPI_ctrl = xr.open_dataarray(f'{path_results}/SST/TPI_ctrl.nc', decode_times=False)
TPI_rcp  = xr.open_dataarray(f'{path_results}/SST/TPI_rcp.nc' , decode_times=False)

In [ ]:
plt.figure(figsize=(12,5))
plt.tick_params(labelsize=14)
plt.axhline(0, c='k', lw=.5)
L1, = plt.plot(Pac_38S_ctrl.time/12+100, chebychev(Pac_38S_ctrl.pcs, 13*12), c='C0', lw=1, ls='--', label='PC 38S')
L2, = plt.plot(Pac_38S_rcp .time/12+300, chebychev(Pac_38S_rcp .pcs, 13*12), c='C1', lw=1, ls='--' )
L3, = plt.plot(Pac_Eq_ctrl .time/12+100, chebychev(Pac_Eq_ctrl .pcs, 13*12), c='C0', lw=1, ls=':' , label='PC Eq.')
L4, = plt.plot(Pac_Eq_rcp  .time/12+300, chebychev(Pac_Eq_rcp  .pcs, 13*12), c='C1', lw=1, ls=':'  )
L5, = plt.plot(Pac_20N_ctrl.time/12+100, chebychev(Pac_20N_ctrl.pcs, 13*12), c='C0', lw=3, ls='-' , label='PC 20N')
L6, = plt.plot(Pac_20N_rcp .time/12+300, -chebychev(Pac_20N_rcp .pcs, 13*12), c='C1', lw=3, ls='-'  )

L7, = plt.plot(TPI_ctrl.time/365     , 5*chebychev(TPI_ctrl, 13), c='C0', lw=2, ls='-.', label='TPI')
L8, = plt.plot(TPI_rcp .time/365-1700, 5*chebychev(TPI_rcp , 13), c='C1', lw=2, ls='-.')

plt.xlabel('time [years]'             , fontsize=16)
plt.ylabel('PDO/IPO/TPI indices', fontsize=16)
plt.legend(handles=[L1, L3, L5, L7],ncol=4, fontsize=16)
plt.tight_layout()
plt.savefig(f'{path_results}/SST/SST_PMV_ctrl_rcp')

## correlation plots

In [ ]:
# %%time
# SST_rect_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_rect_ctrl.nc', decode_times=False)
# SST_rect_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_rect_rcp.nc' , decode_times=False)
# SST_rect_ds_dt_ctrl = lowpass(lowpass(notch(SST_rect_ctrl, 12), 12), 12) - SST_gm_rect_ds_ctrl[:-7]
# SST_rect_ds_dt_rcp  = lowpass(lowpass(notch(SST_rect_rcp , 12), 12), 12) - SST_gm_rect_ds_rcp[:-1]
# SST_rect_ds_dt_ctrl.to_netcdf(f'{path_samoc}/SST/SST_monthly_rect_ds_dt_ctrl.nc')
# SST_rect_ds_dt_rcp .to_netcdf(f'{path_samoc}/SST/SST_monthly_rect_ds_dt_rcp.nc' )

In [ ]:
SST_rect_ds_dt_ctrl = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_rect_ds_dt_ctrl.nc', decode_times=False)
SST_rect_ds_dt_rcp  = xr.open_dataarray(f'{path_samoc}/SST/SST_monthly_rect_ds_dt_rcp.nc' , decode_times=False)

In [ ]:
%%time
# 2:25 min
# ds_20N_ctrl = lag_linregress_3D(Pac_20N_ctrl.pcs[:-7,0], SST_rect_ds_dt_ctrl[24:-(24+7)], dof_corr=1./(12*13))
ds_38S_ctrl = lag_linregress_3D(Pac_38S_ctrl.pcs[:-7,0], SST_rect_ds_dt_ctrl[24:-(24+7)], dof_corr=1./(12*13))
# ds_20N_rcp  = lag_linregress_3D(-Pac_20N_rcp.pcs[:-7,0], SST_rect_ds_dt_rcp [24:-(24+7)], dof_corr=1./(12*13))
ds_38S_rcp  = lag_linregress_3D(Pac_38S_rcp .pcs[:-7,0], SST_rect_ds_dt_rcp [24:-(24+7)], dof_corr=1./(12*13))


In [ ]:
for ds in [ds_20N_ctrl, ds_38S_ctrl]:
    ds.attrs['first_year'] = 102
    ds.attrs['last_year']  = 297
for ds in [ds_20N_rcp, ds_38S_rcp]:
    ds.attrs['first_year'] = 2002
    ds.attrs['last_year']  = 2097

In [ ]:
ds_20N_ctrl

In [ ]:
regr_map(ds=ds_20N_ctrl, index='PDO', run='ctrl', fn=None)

In [ ]:
regr_map(ds=ds_38S_ctrl, index='IPO', run='ctrl', fn=None)

In [ ]:
regr_map(ds=ds_20N_rcp, index='PDO', run='rcp', fn=None)

In [ ]:
regr_map(ds=ds_38S_rcp, index='IPO', run='rcp', fn=None)

In [ ]:
cartopy.__version__